In [1]:
import math
import pymysql
import pandas as pd


import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

import wikipedia
import spacy
from textblob import TextBlob
from IPython.display import Image

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline

from scipy import sparse
from sklearn.metrics import pairwise_distances
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
restaurants_df = pd.read_csv('https://raw.githubusercontent.com/Ajinth/Recommender-System/master/Research/Sample_Restaurants.csv', index_col='Users')

In [3]:
restaurants_df.fillna(value=0, inplace=True)

In [4]:
restaurants_df.head()

,Little_Donkey,Caf__Nero,India_Pavilion,Desi_Dhaba,Thai_Sensation,Dumpling_House,Caf__Luna
Users,,,,,,,
Ajinth,4.0,0.0,5.0,0.0,3.0,0.0,2.0
Tess,3.0,3.0,3.0,0.0,4.0,0.0,0.0
Jessica,4.0,3.0,0.0,0.0,2.0,0.0,1.0
Mark,0.0,2.0,0.0,2.0,0.0,2.0,4.0
Joe,5.0,0.0,5.0,0.0,8.0,0.0,8.0


# Collaborative Item to Item Filtering

In [5]:
# '''List of All Restaurants from the Dataframe'''
# all_restaurants = list(restaurants_df.columns)
# all_restaurants

In [6]:
# '''List of Restaurants for Ajinth'''
# ajinth_restaurant_list = list(restaurants_df.loc['Ajinth', :] ==0)
# ajinth_restaurant_list

In [7]:
# '''List of Restaurants that Ajinth has Rated'''
# user_rated=[]
# for key, a_res in enumerate(ajinth_restaurant_list):
#     if a_res == False:
#         user_rated.append((restaurants_df.loc['Ajinth', :] ==0).index[key])
#     else: 
#         continue
# print (user_rated)  

In [8]:
'''Function to return the list of restaurants rated by the user'''
def user_rated_list(user):
    user_rated=[]
    for key, a_res in enumerate(ajinth_restaurant_list):
        if a_res == False:
            user_rated.append((restaurants_df.loc[user, :] ==0).index[key])
        else: 
            continue
    return user_rated   

In [9]:
user = 'Ajinth'
user_rated_list(user)

NameError: name 'ajinth_restaurant_list' is not defined

In [ ]:
# '''List of Restaurants that Ajinth has Not Rated'''
# user_not_rated=[]
# for key, a_res in enumerate(ajinth_restaurant_list):
#     if a_res == True:
#         user_not_rated.append((restaurants_df.loc['Ajinth', :] ==0).index[key])
#     else: 
#         continue
# print (user_not_rated) 

In [ ]:
'''Function to return the list of restaurants yet to be rated by the user'''
def user_not_rated_list(user):
    user_not_rated=[]
    for key, a_res in enumerate(ajinth_restaurant_list):
        if a_res == True:
            user_not_rated.append((restaurants_df.loc['Ajinth', :] ==0).index[key])
        else: 
            continue
    return (user_not_rated) 

In [ ]:
user = 'Ajinth'
user_not_rated_list(user)

In [ ]:
# (restaurants_df.loc['Ajinth', :] ==0).index[0]

In [ ]:
'''Custom Function to calculate the Co-Similarity'''
def cosim(v1, v2):
    dot_product = sum(n1 * n2 for n1, n2 in zip(v1, v2) )
    magnitude1 = math.sqrt(sum(n ** 2 for n in v1))
    magnitude2 = math.sqrt(sum(n ** 2 for n in v2))
    return dot_product / (magnitude1 * magnitude2)

In [ ]:
# '''
# Looping through the User Restaurants and all restaurants to calculate the Matrix.
# The results are stored in a dictionary
# '''
# ur_rest_dict = {}
# for ur in user_rated: 
#     ur_cosim=[]
#     for ar in all_restaurants: 
#         similarity = cosim(restaurants_df[ur],\
#                                         restaurants_df[ar])
#         ur_cosim.append(similarity)
#     ur_rest_dict[ur] = ur_cosim
# # print (ur_rest_dict)

In [ ]:
'''This function calcultes the co-sine similarity between the restaurants rsted by the user and all the restaurants in the list'''
def cooccurence_matrix_creator():
    '''Calling the user rated function'''
    user_rated_list(user)
    
    ur_rest_dict = {}
    for ur in user_rated: 
        ur_cosim=[]
        for ar in all_restaurants: 
            similarity = cosim(restaurants_df[ur],\
                                            restaurants_df[ar])
            ur_cosim.append(similarity)
        ur_rest_dict[ur] = ur_cosim
    co_occurence_matrix = pd.DataFrame.from_dict(ur_rest_dict, orient='index')
    co_occurence_matrix.columns=all_restaurants
    return co_occurence_matrix

In [ ]:
co_occurence_matrix.head()

In [ ]:
# '''Dataframe for the co-occurence matrix'''
# co_occurence_matrix = pd.DataFrame.from_dict(ur_rest_dict, orient='index')
# co_occurence_matrix.columns=all_restaurants
# co_occurence_matrix

# Review of some of the Variables

In [ ]:
user_rated

In [ ]:
user_not_rated

In [ ]:
co_occurence_matrix

In [ ]:
restaurants_df

In [ ]:
# type(restaurants_df.loc['Ajinth', : ])

In [ ]:
# restaurants_to_rate = co_occurence_matrix[user_not_rated_list(user)]
# restaurants_to_rate

In [ ]:
def rest_to_rate(df):
    restaurants_to_rate = df[user_not_rated_list(user)]
    return restaurants_to_rate

In [ ]:
rest_to_rate(co_occurence_matrix)

In [ ]:
'''Calculating Predicted Score for Cafe Nero - Manual Method'''
((0.64*4)+(.17*5) + (0.27*3) + (.17*2))/(0.64+0.17+0.27+0.17)

In [ ]:
'''Calculating Predicted Score for Dumpling House - Manual Method'''
((0.309839*4)+(2*0.194029))/(0.309839+0.194029)

In [ ]:
# '''Calculating the Predicted Score'''

# predicted_rating_dict = {}
# for not_rated in restaurants_to_rate.columns: 
#     column_df = pd.DataFrame(restaurants_to_rate[not_rated].sort_values(ascending=False))
#     predicted_rating = []
#     top_sim_list = []
#     for top_n in range(2):   # Just getting the top two rated items 
        
#         top_sim = column_df[not_rated][top_n]
#         top_sim_list.append(top_sim)
        
#         given_rr = column_df[not_rated].index[top_n]
#         given_rank_df = pd.DataFrame(restaurants_df.loc['Ajinth', :]) # While converting this to function the name should be an argument to the function
#         given_rank = given_rank_df.loc[given_rr,:]
        
#         predicted_rating_item = float(given_rank*top_sim)
#         predicted_rating.append(predicted_rating_item)
#     total_sim_weight = sum(top_sim_list)
#     adjusted_predicted_rating = (sum(predicted_rating))/total_sim_weight
#     predicted_rating_dict[not_rated] = adjusted_predicted_rating
# print (predicted_rating_dict)
    



In [ ]:
'''Generating the prediction'''


def rating_predictor_colab_ii():
    predicted_rating_dict = {}
    for not_rated in restaurants_to_rate.columns: 
        column_df = pd.DataFrame(restaurants_to_rate[not_rated].sort_values(ascending=False))
        predicted_rating = []
        top_sim_list = []
        for top_n in range(2):   # Just getting the top two rated items 

            top_sim = column_df[not_rated][top_n]
            top_sim_list.append(top_sim)

            given_rr = column_df[not_rated].index[top_n]
            given_rank_df = pd.DataFrame(restaurants_df.loc[user, :]) # While converting this to function the name should be an argument to the function
            given_rank = given_rank_df.loc[given_rr,:]

            predicted_rating_item = float(given_rank*top_sim)
            predicted_rating.append(predicted_rating_item)
        total_sim_weight = sum(top_sim_list)
        adjusted_predicted_rating = (sum(predicted_rating))/total_sim_weight
        predicted_rating_dict[not_rated] = adjusted_predicted_rating
        
    '''Creating the Dataframe in this step'''
        
    predicted_df = pd.DataFrame.from_dict(predicted_rating_dict, orient='index')
    predicted_df.columns=[user]
    return predicted_df
#     return (predicted_rating_dict)


In [ ]:
predicted_df

In [ ]:
# predicted_df = pd.DataFrame.from_dict(predicted_rating_dict, orient='index')
# predicted_df.columns=['Ajinth']
# predicted_df

In [ ]:
#End